In [31]:
import pandas as pd

df = pd.read_csv("1950big_data.csv", low_memory=False) 
print(df.shape)
df.info()


(455908, 36)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455908 entries, 0 to 455907
Data columns (total 36 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   year                455908 non-null  int64  
 1   position            4028 non-null    float64
 2   title               455901 non-null  object 
 3   artist              4028 non-null    object 
 4   pos_sentiment       4028 non-null    float64
 5   neg_sentiment       4028 non-null    float64
 6   neut_sentiment      4028 non-null    float64
 7   compound_sentiment  4028 non-null    float64
 8   f_k_grade           4028 non-null    float64
 9   flesch_index        4028 non-null    float64
 10  fog_index           4028 non-null    float64
 11  num_syllables       4028 non-null    float64
 12  difficult_words     4028 non-null    float64
 13  num_dupes           4028 non-null    float64
 14  num_words           4028 non-null    float64
 15  num_lines           4

In [32]:
na_count = df.isna().sum().sort_values(ascending=False)
na_percent = (na_cnt / len(df)).sort_values(ascending=False)
missing = pd.concat([na_count, na_percent], axis=1)
missing.columns = ["na_count", "na_percent"]
missing.head(20)
# 

,na_count,na_percent
genre_tags,453236,0.994139
difficult_words,451880,0.991165
f_k_grade,451880,0.991165
num_lines,451880,0.991165
num_words,451880,0.991165
num_dupes,451880,0.991165
num_syllables,451880,0.991165
fog_index,451880,0.991165
flesch_index,451880,0.991165
compound_sentiment,451880,0.991165


A subset of variables exhibits extremely high levels of missingness, with more than 99% of observations missing. These variables share a common characteristic: they rely on external information sources beyond the core Spotify audio metadata. Specifically, this group includes genre-related information (genre_tags), lyric-based textual features (such as num_words, num_lines, num_syllables, f_k_grade, fog_index, flesch_index, and difficult_words), sentiment analysis scores (pos_sentiment, neg_sentiment, neut_sentiment, and compound_sentiment), as well as Billboard-specific metadata (artist and position).

The high missingness in these variables does not indicate data quality errors but rather reflects limited coverage from external data sources, including lyric databases, genre classification pipelines, and historical Billboard ranking records. Only a small subset of songs could be successfully matched to these auxiliary sources. Consequently, in a large-scale dataset comprising over 450,000 tracks, missingness exceeding 99% for these variables is an expected outcome of incomplete data integration rather than a result of incorrect data processing.

To mitigate potential bias and ensure robustness in exploratory analysis, these variables were retained in the raw dataset for documentation purposes but excluded from the primary analytical dataset used for exploratory data analysis and feature engineering.

In [33]:
## core variable
core = ["danceability","energy","valence","tempo","loudness","acousticness",
"speechiness","instrumentalness","liveness","duration_ms","popularity"]

## These variables are not necessarily provided by Spotify itself, nor are they directly derived from the audio signal, 
## and therefore must be supplemented using external data sources.
extra = ["genre_tags","num_words","num_lines","num_syllables","flesch_index",
          "fog_index","f_k_grade","difficult_words","pos_sentiment","neg_sentiment",
          "neut_sentiment","compound_sentiment","position"]

print(df[core].isna().mean().sort_values(ascending=False))
print(df[extra].isna().mean().sort_values(ascending=False))


danceability        0.005554
energy              0.005554
valence             0.005554
tempo               0.005554
loudness            0.005554
acousticness        0.005554
speechiness         0.005554
instrumentalness    0.005554
liveness            0.005554
duration_ms         0.005554
popularity          0.005554
dtype: float64
genre_tags            0.994139
num_words             0.991165
num_lines             0.991165
num_syllables         0.991165
flesch_index          0.991165
fog_index             0.991165
f_k_grade             0.991165
difficult_words       0.991165
pos_sentiment         0.991165
neg_sentiment         0.991165
neut_sentiment        0.991165
compound_sentiment    0.991165
position              0.991165
dtype: float64


In [34]:
thr = 0.95
drop_cols = missing.index[missing["na_percent"] > thr].tolist()

analysis_df = df.drop(columns=drop_cols)
print("analysis_df shape:", analysis_df.shape)
analysis_df.info()

analysis_df shape: (455908, 21)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 455908 entries, 0 to 455907
Data columns (total 21 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   year              455908 non-null  int64  
 1   title             455901 non-null  object 
 2   id                453376 non-null  object 
 3   popularity        453376 non-null  float64
 4   duration_ms       453376 non-null  float64
 5   explicit          453376 non-null  float64
 6   artists           453376 non-null  object 
 7   id_artists        453376 non-null  object 
 8   danceability      453376 non-null  float64
 9   energy            453376 non-null  float64
 10  key               453376 non-null  float64
 11  loudness          453376 non-null  float64
 12  mode              453376 non-null  float64
 13  speechiness       453376 non-null  float64
 14  acousticness      453376 non-null  float64
 15  instrumentalness  453376 non-null  f

In [35]:
analysis_df.isna().mean().sort_values(ascending=False).head(10)
# There is no severe or huge missing value

key                 0.005554
time_signature      0.005554
valence             0.005554
liveness            0.005554
instrumentalness    0.005554
acousticness        0.005554
speechiness         0.005554
mode                0.005554
loudness            0.005554
energy              0.005554
dtype: float64

Given the extremely low proportion of missing values and the absence of a clear, domain-justified similarity metric among songs, median imputation was chosen over KNN-based methods to avoid introducing unnecessary model-driven assumptions and potential noise.

More sophisticated imputation methods, such as KNN-based imputation, were considered but not adopted, as their assumptions were not aligned with the exploratory focus of this project.

In [36]:
num_cols = analysis_df.select_dtypes(include="number").columns
for c in num_cols:
     analysis_df[c] = analysis_df[c].fillna(analysis_df[c].median())


In [42]:
analysis_df.isna().sum()

year                   0
title                  7
id                  2532
popularity             0
duration_ms            0
explicit               0
artists             2532
id_artists          2532
danceability           0
energy                 0
key                    0
loudness               0
mode                   0
speechiness            0
acousticness           0
instrumentalness       0
liveness               0
valence                0
tempo                  0
time_signature         0
hit                    0
dtype: int64

A small number of missing values remain in identifier and textual fields such as title, artists, id, and id_artists. These variables serve as descriptive identifiers rather than analytical features. Since imputing such fields would introduce artificial or misleading information, missing values in these columns were retained and excluded from numerical preprocessing.